In [7]:
! pip install IMDbPY


     |################################| 307kB 1.0MB/s eta 0:00:01


In [13]:
from pathlib import Path

In [14]:
data_dir = Path("data")

In [15]:
from iso639 import languages

In [20]:
langs = [l.alpha2 for l in languages if l.alpha2]

In [21]:
langs_str = ",".join(langs)

In [55]:
!ls -lR data/posters/ | grep .jpg | wc -l

132354


In [70]:
!ls -lR data/posters/ | grep .json | wc -l

10445


In [57]:
!ls -lR data/posters/ | grep .png | wc -l

16


In [72]:
132354+10445*2+16

153260

In [168]:
!ls   data/posters/Zootopia

1F8y4qwbVTWv6yI7jVaVGyubn9b.jpg  kyiEmCk6mOenJiL7s3LspVKkoZM.jpg
1vDl7yuBZBBgTGSOmW07aELAl4c.jpg  lCCkDggLS1dvPwkUUvxdExFTvVa.jpg
3BISFHsKQawmAk8yrbbZxVRPotR.jpg  lfvETUFUxzfvUPeC7hKtWdoS8C4.jpg
3mXuLurmIFmROfyrh5ANANuVrfF.jpg  m7jHtpiSXo4XpL3A9JAUriT7s4K.jpg
3pu7fvygjLrArtxrSzMjKBBrGe5.jpg  mBQBrTt2otVfcVDBxYyoqPNexrg.jpg
41ylKn1W9EBSr3ZneESz5tFDLbc.jpg  mKfOGVyKMRliFHAlTMKz7zNKuTJ.jpg
4PVshNU3dqNdAKQyT4pW0TjTL2N.jpg  mSgVzSKgAArXtxIZ6PZF95rWdK9.jpg
4appyqceHISe330r4zDZcV1x0wT.jpg  mV5jB8d6RDaI8unN1NJiKuQEExG.jpg
5dmPVuL9IvcmpGAnUWUEMKfHFTP.jpg  mYzoH5xEueCP6uHfbjFGUNJBRyg.jpg
5gezpRVxHQlMohOHKtgvFLVcxyA.jpg  metadata.json
6QpFezvfMO2luiKBblRnjqE4VaB.jpg  mwpG2dQMfSaO61Je53ooU5xJDin.jpg
7H9mPapkjHgsmBll9PhOWE3usJy.jpg  nZAnKuEhztoOFi8PSUMPUcFLn2u.jpg
7X2CZ3SRCrikbLnGVUSgSghXuGV.jpg  obKkFXDptJlSi2TGUav0u4xBkxO.jpg
84VKdXaz6W3QF9yWYYQbCyibdys.jpg  pIsn2LIbeyxf6zP1RPKcb89pPlh.jpg
9AbPONTzDy1rO6brklSbNjoJOlK.jpg  pNGvlksI0uAPY2YdLKHc1tJmzEG.jpg
9LRFNLthb0jfWC0HUDUWjYgpP9F.jpg  qWmTDH2FHx

In [65]:
!ls -l data/posters/  | tail -10

drwxrwx--- 2 kagandi@auth.ad.bgu.ac.il kagandi@auth.ad.bgu.ac.il  4096 Nov 17 16:25 Zona Zamfirova
drwxrwx--- 2 kagandi@auth.ad.bgu.ac.il kagandi@auth.ad.bgu.ac.il  4096 Nov 16 15:39 Zootopia
drwxrwx--- 2 kagandi@auth.ad.bgu.ac.il kagandi@auth.ad.bgu.ac.il  4096 Nov 17 00:00 Zorba the Greek
drwxrwx--- 2 kagandi@auth.ad.bgu.ac.il kagandi@auth.ad.bgu.ac.il  4096 Nov 18 13:55 Zorro
drwxrwx--- 2 kagandi@auth.ad.bgu.ac.il kagandi@auth.ad.bgu.ac.il  4096 Nov 18 17:06 Zorro: The Gay Blade
drwxrwx--- 2 kagandi@auth.ad.bgu.ac.il kagandi@auth.ad.bgu.ac.il  4096 Nov 18 12:09 Zu: Warriors from the Magic Mountain
drwxrwx--- 2 kagandi@auth.ad.bgu.ac.il kagandi@auth.ad.bgu.ac.il  4096 Nov 16 23:59 Zulu
drwxrwx--- 2 kagandi@auth.ad.bgu.ac.il kagandi@auth.ad.bgu.ac.il  4096 Nov 18 06:43 Zulu Dawn
drwxrwx--- 2 kagandi@auth.ad.bgu.ac.il kagandi@auth.ad.bgu.ac.il  4096 Nov 18 05:43 eXistenZ
drwxrwx--- 2 kagandi@auth.ad.bgu.ac.il kagandi@auth.ad.bgu.ac.il  4096 Nov 18 16:59 ffolkes


In [69]:
!ls data

curr_id.txt  posters  reduced-ids.csv


In [51]:
!ls -l data/posters/ | wc -l

10464


In [52]:
184713 - 10464

174249

In [ ]:
import imdb
import tmdbsimple as tmdb
import pandas as pd
import requests
from PIL import Image
from io import BytesIO
import os
from tqdm import tqdm_notebook as tqdm
import json

imdbPy = imdb.IMDb()
tmdb.API_KEY = 'ed22c580a935db103cea17eb93c25567'
url = 'https://image.tmdb.org/t/p/w780'

posters_dir = data_dir / 'posters'

ids = pd.read_csv(data_dir/'reduced-ids.csv', header=0)

def save_img(dir_path, save_path, url):
  response = requests.get(f'{url}')
  img = Image.open(BytesIO(response.content))

  if not os.path.exists(dir_path):
    os.makedirs(dir_path)

  img.save(f'{dir_path}/{save_path}')

with (data_dir/'curr_id.txt').open("r") as f:
    curr = int(f.read())
print(curr)
for i, m_id in enumerate(tqdm(ids['tconst'][curr:])):
    try:
        number_id = m_id.partition('tt')[2]
        m = imdbPy.get_movie(number_id)
        dir_path = posters_dir/m["title"]

        if "full-size cover url" in m.keys():
            save_img(dir_path, f'{m_id}-imdbpy.jpg', m["full-size cover url"])
            print(f'saved {m_id} - {m["title"]}')
        else:
            print(f'{m_id} does not have a poster')

#         print('tmdb')
        imgs_metadata = tmdb.Movies(m_id).images()['posters']
        for img in imgs_metadata:
            save_img(dir_path, f'{img["file_path"][1:]}', f'{url}{img["file_path"]}')
#             print(f'saved {img["file_path"][1:]} - {m["title"]}')
        with (dir_path/'metadata.json').open("w") as f:
            json.dump(imgs_metadata, f)
        with (data_dir/'curr_id.txt').open("w") as f:
            f.write(f"{curr+i}")
    except UnicodeEncodeError:
        pass
    except OSError:
        pass
    except imdb.IMDbDataAccessError:
        pass
    except KeyError:
        pass

9069


saved tt0199777 - Moloch
saved tt0116410 - Ghosts of Mississippi
saved tt0116442 - Grace of My Heart
saved tt0105378 - Shadows and Fog
saved tt0269389 - Jalla! Jalla!
saved tt0105415 - Singles
saved tt0097858 - Meet the Feebles
saved tt0270933 - Escaflowne: The Movie
saved tt0118113 - Walking and Talking
saved tt0117438 - Ransom
saved tt0118764 - Les Boys
saved tt0116242 - Everyone Says I Love You
saved tt0118736 - Six-String Samurai
saved tt0196216 - Small Time Crooks
saved tt0120835 - A Soldier's Daughter Never Cries
saved tt0286106 - Signs
saved tt0117509 - Romeo + Juliet
saved tt0118636 - Apt Pupil
saved tt0116324 - Flirting with Disaster
saved tt0265713 - Scotland, Pa.
saved tt0116683 - James and the Giant Peach
saved tt0116999 - Marvin's Room
saved tt0105572 - That Night
saved tt0280990 - The Nest
saved tt0104321 - Gas Food Lodging
saved tt0189142 - Marcel Proust's Time Regained
saved tt0105789 - The Waterdance
saved tt0190138 - The Whole Nine Yards
saved tt0118548 - Absolute Pow

2020-11-18 13:31:28,519 CRITICAL [imdbpy] /storage/homedir/dima/miniconda3/envs/subs2graph/lib/python3.6/site-packages/imdb/_exceptions.py:34: IMDbDataAccessError exception raised; args: ({'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/title/tt0116163/reference', 'proxy': '', 'exception type': 'IOError', 'original exception': timeout('The read operation timed out',)},); kwds: {}
Traceback (most recent call last):
  File "/storage/homedir/dima/miniconda3/envs/subs2graph/lib/python3.6/site-packages/imdb/parser/http/__init__.py", line 220, in retrieve_unicode
    response = uopener.open(url)
  File "/storage/homedir/dima/miniconda3/envs/subs2graph/lib/python3.6/urllib/request.py", line 526, in open
    response = self._open(req, data)
  File "/storage/homedir/dima/miniconda3/envs/subs2graph/lib/python3.6/urllib/request.py", line 544, in _open
    '_open', req)
  File "/storage/homedir/dima/miniconda3/envs/subs2graph/lib/python3.6/urllib/request.py", line 504, in _call_cha

saved tt0116136 - DragonHeart
saved tt0105690 - Under Siege
saved tt0077531 - F.I.S.T.
saved tt0053825 - Flaming Star
saved tt0071276 - Captain Kronos: Vampire Hunter
saved tt0120347 - Tomorrow Never Dies
saved tt0053877 - The Grass Is Greener
saved tt0066402 - Start the Revolution Without Me
saved tt0119577 - Love Is the Devil: Study for a Portrait of Francis Bacon
saved tt0071396 - Messiah of Evil
saved tt0053882 - The 3 Worlds of Gulliver
saved tt0071517 - Foxy Brown
saved tt0053644 - The Bellboy
saved tt0070819 - A Touch of Class
saved tt0070825 - The Train Robbers
saved tt0066518 - The Vampire Lovers
saved tt0066495 - The Twelve Chairs
saved tt0112571 - Boys on the Side
saved tt0049815 - The Swan
saved tt0071571 - Gone in 60 Seconds
saved tt0109255 - Maybe... Maybe Not
saved tt0065769 - The Troops on Vacation
saved tt0072448 - Zorro
saved tt0072653 - The Apple Dumpling Gang
saved tt0054188 - Please Don't Eat the Daisies
saved tt0112792 - Dangerous Minds
saved tt0071706 - Juggernau

In [39]:
m["full-size cover url"]

'https://m.media-amazon.com/images/M/MV5BNmQwNjAzZTYtOWM1OS00YTQ3LWJlYjAtY2Y3ZmJmNDgyNzM5XkEyXkFqcGdeQXVyMjQ2OTI2MzU@.jpg'

In [34]:
import json


In [40]:
dir_path

PosixPath('data/posters/Süt Kardesler')

In [74]:
posters_dir.iterdir()

<generator object Path.iterdir at 0x7f0724223af0>

In [176]:
import json
movie_posters = []
for movie in tqdm(posters_dir.iterdir(), total=10464):
    try:
        with (movie/"metadata.json").open("r") as f:
            movie_metadata = json.load(f)
            df = pd.DataFrame(movie_metadata)
            tconst = list(movie.glob("*imdbpy.jpg"))[0].name.split("-")[0]
            df["movie"] = movie.name
            df["tconst"] = tconst
            movie_posters.append(df)
    except FileNotFoundError:
        print(movie.name)


C
Little Forest: Summer
Up in Smoke
F
Appleseed
50
25 km
Wahed men el nas
A Love Song for Bobby Long
Hitchcock
Marat
Auntie Danielle
Deja Vu
Zabriskie Point
I Love You, Now Die: The Commonwealth v. Michelle Carter
Courage & Stupidity
78
Black Legion
Can't Complain
Priceless
Bros.
The Shining
Bullets or Ballots
Fahrenheit 9
Naked Gun 33 1
An Hour to Kill
Leaving Neverland
Frost
Dhruva
East
7
The Embalmer
The Bugs Bunny


In [174]:
tconst

'tt6679360'

In [177]:
movie_posters = pd.concat(movie_posters)

/storage/homedir/dima/miniconda3/envs/subs2graph/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [106]:
movie_posters["iso_639_1"].value_counts().head(20)

en    40549
fr    10958
es     9057
de     8558
it     7809
pt     5803
ko     3682
ru     3673
hu     3500
pl     2319
ja     2199
uk     1578
bg     1477
zh     1474
he     1217
cs     1095
sv     1041
da      888
tr      717
el      652
Name: iso_639_1, dtype: int64

In [93]:
!ls -lh  data/posters/Black\ Legion


total 816K
-rw-rw---- 1 kagandi@auth.ad.bgu.ac.il kagandi@auth.ad.bgu.ac.il  67K Nov 17 23:47 9g8F2u5sn2DvFovY1Dxqyd1Ryvi.jpg
-rw-rw---- 1 kagandi@auth.ad.bgu.ac.il kagandi@auth.ad.bgu.ac.il 737K Nov 17 23:47 tt0027367-imdbpy.jpg


In [178]:
movie_posters = movie_posters.reset_index(drop=True)

In [179]:
movie_posters.to_pickle("movie_posters.pkl")

In [180]:
movie_posters

,aspect_ratio,file_path,height,iso_639_1,movie,tconst,vote_average,vote_count,width
0,0.666667,/7Sd8ALunb5y3GOqX760Jd2rFPN7.jpg,975.0,ru,On vam ne Dimon,tt6679360,5.312,1.0,650.0
1,0.707000,/kI9NRfC9CdPmL7YzE3czdClKRmA.jpg,1000.0,ru,On vam ne Dimon,tt6679360,5.172,1.0,707.0
2,0.666667,/cmDLj7JTZkcBl5HHAQopv2s9o9A.jpg,891.0,None,Visaaranai,tt4991384,0.000,0.0,594.0
3,0.666667,/8HbknP82NFye7GCsL8L9vyWzMUM.jpg,750.0,en,Visaaranai,tt4991384,0.000,0.0,500.0
4,0.666667,/gHJLb6NGyqSHdzTPWewSBObidNI.jpg,1500.0,ru,Office Romance,tt0076727,0.000,0.0,1000.0
...,...,...,...,...,...,...,...,...,...
117200,0.666667,/8jFp8nlyJ84J38YBwFilPmyXiXj.jpg,1440.0,it,The Turning Point,tt0076843,0.000,0.0,960.0
117201,0.666667,/tg3GaMqXx3SNaxSUT1PE8Srt3Wl.jpg,750.0,en,The Turning Point,tt0076843,0.000,0.0,500.0
117202,0.666667,/9jujXWhUH6uwl8IPMq5XrzV12hZ.jpg,1500.0,de,The Turning Point,tt0076843,0.000,0.0,1000.0
117203,0.705128,/cf1ZIx7zTHYcDWZRGIvsZPm1FbB.jpg,780.0,ja,The Turning Point,tt0076843,0.000,0.0,550.0


In [195]:
df = pd.read_csv(data_dir/"reduced-merged.csv")

In [196]:
df.genres = df.genres.str.split(",")

In [197]:
import numpy as np


In [198]:
df["genres"] = df["genres"].apply(lambda x: [] if  x is np.nan else x)

In [200]:
movie_posters[movie_posters["movie"]=="Asterix and Cleopatra"]

,aspect_ratio,file_path,height,iso_639_1,movie,tconst,vote_average,vote_count,width
94028,0.666667,/gNpKAmWFsCiWKKx39Ap8pF7EWQg.jpg,1200.0,en,Asterix and Cleopatra,tt0062687,5.522,4.0,800.0
94029,0.710480,/mu6cDID2uSCGboiEmT6ikW9dO30.jpg,2252.0,de,Asterix and Cleopatra,tt0062687,5.312,1.0,1600.0
94030,0.666667,/rN7OqATGLDlIdJGfvocmm9wZ6fO.jpg,1050.0,fr,Asterix and Cleopatra,tt0062687,5.312,1.0,700.0
94031,0.666667,/fsvNagTYR9n8Edc60fdXffIU1rJ.jpg,750.0,pt,Asterix and Cleopatra,tt0062687,5.312,1.0,500.0
94032,0.666667,/jBwKVkutN1A1ViQL3xwSqeM4bz6.jpg,1500.0,de,Asterix and Cleopatra,tt0062687,5.312,1.0,1000.0
94033,0.666667,/xdmIk7JjMXwYIkh1k3TIG8eO2h2.jpg,2100.0,da,Asterix and Cleopatra,tt0062687,5.312,1.0,1400.0
94034,0.698667,/3RnFexw6V3vml0xKuFqR2xYKuKH.jpg,750.0,sv,Asterix and Cleopatra,tt0062687,5.312,1.0,524.0
94035,0.666667,/udrubAPoLPIqcfwzBJPukiVp9Qs.jpg,1050.0,fr,Asterix and Cleopatra,tt0062687,5.246,2.0,700.0
94036,0.666667,/wSHK5YEHpBsZDKDdKnz3XVvwHCk.jpg,1500.0,fr,Asterix and Cleopatra,tt0062687,5.246,2.0,1000.0
94037,0.666667,/q5wzVXmaNyCeP1Y4oLbs1ONnSTK.jpg,900.0,fr,Asterix and Cleopatra,tt0062687,5.246,2.0,600.0


In [202]:
df[df["tconst"]=="tt0062687"]

,tconst,averageRating,numVotes,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
31808,tt0062687,7.2,11670,movie,Asterix & Cleopatra,Astérix et Cléopâtre,0,1968.0,NaN,72.0,"[Adventure, Animation, Comedy]"


In [203]:
df2  = df.merge(movie_posters,  on="tconst")

In [204]:
df2

,tconst,averageRating,numVotes,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,aspect_ratio,file_path,height,iso_639_1,movie,vote_average,vote_count,width
0,tt0002130,7.1,2289,movie,Dante's Inferno,L'Inferno,0,1911.0,NaN,71.0,"[Adventure, Drama, Fantasy]",0.666667,/ky0z9hDlajlwL3fBqwsCSbjJGjR.jpg,2100.0,en,Dante's Inferno,5.454,3.0,1400.0
1,tt0002130,7.1,2289,movie,Dante's Inferno,L'Inferno,0,1911.0,NaN,71.0,"[Adventure, Drama, Fantasy]",0.666667,/sL1ovbqIKapcyGeQ3ytBz919uAa.jpg,2100.0,en,Dante's Inferno,5.246,2.0,1400.0
2,tt0002130,7.1,2289,movie,Dante's Inferno,L'Inferno,0,1911.0,NaN,71.0,"[Adventure, Drama, Fantasy]",0.666667,/kV1vTVDxQBJR2ezUqlviUXbFb3r.jpg,1500.0,en,Dante's Inferno,5.172,1.0,1000.0
3,tt0002130,7.1,2289,movie,Dante's Inferno,L'Inferno,0,1911.0,NaN,71.0,"[Adventure, Drama, Fantasy]",0.666667,/pvtyhAxKNM0tJbE1PbayAqR6lCl.jpg,750.0,it,Dante's Inferno,0.000,0.0,500.0
4,tt0003740,7.1,3109,movie,Cabiria,Cabiria,0,1914.0,NaN,148.0,"[Adventure, Drama, History]",0.666667,/oZhgcHHapkzYmB0OhQcRfZ7LdLM.jpg,3000.0,en,Cabiria,5.388,4.0,2000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117200,tt9900782,8.5,9353,movie,Kaithi,Kaithi,0,2019.0,NaN,145.0,"[Action, Thriller]",0.666667,/hOF9CgPsy9aLr5GJEBESC8MEXFy.jpg,1200.0,ta,Kaithi,5.384,2.0,800.0
117201,tt9900782,8.5,9353,movie,Kaithi,Kaithi,0,2019.0,NaN,145.0,"[Action, Thriller]",0.732000,/fkfOz5UUnQ87XD71jWYGCXAl1Yf.jpg,1000.0,None,Kaithi,0.000,0.0,732.0
117202,tt9900782,8.5,9353,movie,Kaithi,Kaithi,0,2019.0,NaN,145.0,"[Action, Thriller]",0.666667,/uMuCsmLVj8HuOwm5P8RTo8m3qmG.jpg,1140.0,ta,Kaithi,0.000,0.0,760.0
117203,tt9900782,8.5,9353,movie,Kaithi,Kaithi,0,2019.0,NaN,145.0,"[Action, Thriller]",0.666667,/gSuzfy4nYkJbzfQXRufI0fU6Ph9.jpg,1020.0,ta,Kaithi,0.000,0.0,680.0


In [205]:
df2["genres"] = df2["genres"].apply(lambda x: [] if  x is np.nan else x)

In [206]:
df2 = df2[df2["genres"].apply(lambda x: "Animation" not in x)]

In [209]:
df2.to_pickle("movie_posters_filtered.pkl")

In [ ]:
!zip -rq posters.zip data/